In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [4]:
import sys
sys.path.insert(0, "/home/husein/parsing/self-attentive-parser/src")
sys.path.append("/home/husein/parsing/self-attentive-parser")

In [5]:
import tensorflow as tf
from transformers import AlbertTokenizer

In [6]:
from transformers import AlbertTokenizer
tokenizer = AlbertTokenizer.from_pretrained(
    'huseinzol05/bert-base-bahasa-cased',
    unk_token = '[UNK]',
    pad_token = '[PAD]',
    do_lower_case = False,
)

In [33]:
tokenizer.__dict__

{'init_inputs': (),
 'init_kwargs': {'unk_token': '[UNK]',
  'pad_token': '[PAD]',
  'do_lower_case': False,
  'vocab_file': '/home/husein/.cache/torch/transformers/0c9fbc56f2a8aa995421aacf12696a100db75b9709be6c10a837750077f21259.44cfb00375f2ad0390cef8e4d6b99043bba75c348ba6805ae44e5a4a5b59ebfa',
  'special_tokens_map_file': '/home/husein/.cache/torch/transformers/3537099277b83ccb202b83879ac7c8594c862168b7dfffb15a42881cde2e50c8.1f04d662a14dbb60dcf1073d308fd370f7244f21fee819dd5627fb5afeb761b1',
  'full_tokenizer_file': None},
 'model_max_length': 1000000000000000019884624838656,
 'padding_side': 'right',
 'model_input_names': ['token_type_ids', 'attention_mask'],
 '_bos_token': '[CLS]',
 '_eos_token': '[SEP]',
 '_unk_token': '[UNK]',
 '_sep_token': '[SEP]',
 '_pad_token': '[PAD]',
 '_cls_token': '[CLS]',
 '_mask_token': '[MASK]',
 '_pad_token_type_id': 0,
 '_additional_special_tokens': [],
 'verbose': True,
 'added_tokens_encoder': {},
 'added_tokens_decoder': {},
 'unique_no_split_token

In [32]:
tokenizer.sp_model

['DecodeIds',
 'DecodeIdsAsSerializedProto',
 'DecodePieces',
 'DecodePiecesAsSerializedProto',
 'EncodeAsIds',
 'EncodeAsPieces',
 'EncodeAsSerializedProto',
 'GetPieceSize',
 'GetScore',
 'IdToPiece',
 'IsControl',
 'IsUnknown',
 'IsUnused',
 'Load',
 'LoadFromSerializedProto',
 'LoadOrDie',
 'LoadVocabulary',
 'NBestEncodeAsIds',
 'NBestEncodeAsPieces',
 'NBestEncodeAsSerializedProto',
 'PieceToId',
 'ResetVocabulary',
 'SampleEncodeAsIds',
 'SampleEncodeAsPieces',
 'SampleEncodeAsSerializedProto',
 'SetDecodeExtraOptions',
 'SetEncodeExtraOptions',
 'SetVocabulary',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_dest

In [7]:
import json

with open('vocab.json') as fopen:
    data = json.load(fopen)
    
LABEL_VOCAB = data['label']
TAG_VOCAB = data['tag']

In [8]:
with tf.gfile.GFile('export/model.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

In [9]:
input_ids = graph.get_tensor_by_name('import/input_ids:0')
word_end_mask = graph.get_tensor_by_name('import/word_end_mask:0')
charts = graph.get_tensor_by_name('import/charts:0')
tags = graph.get_tensor_by_name('import/tags:0')
sess = tf.InteractiveSession(graph = graph)

In [10]:
BERT_MAX_LEN = 512
import numpy as np
from parse_nk import BERT_TOKEN_MAPPING

def make_feed_dict_bert(sentences):
    all_input_ids = np.zeros((len(sentences), BERT_MAX_LEN), dtype=int)
    all_word_end_mask = np.zeros((len(sentences), BERT_MAX_LEN), dtype=int)
    

    subword_max_len = 0
    for snum, sentence in enumerate(sentences):
        tokens = []
        word_end_mask = []

        tokens.append(u"[CLS]")
        word_end_mask.append(1)

        cleaned_words = []
        for word in sentence:
            word = BERT_TOKEN_MAPPING.get(word, word)
            # BERT is pre-trained with a tokenizer that doesn't split off
            # n't as its own token
            if word == u"n't" and cleaned_words:
                cleaned_words[-1] = cleaned_words[-1] + u"n"
                word = u"'t"
            cleaned_words.append(word)

        for word in cleaned_words:
            word_tokens = tokenizer.tokenize(word)
            if not word_tokens:
                # The tokenizer used in conjunction with the parser may not
                # align with BERT; in particular spaCy will create separate
                # tokens for whitespace when there is more than one space in
                # a row, and will sometimes separate out characters of
                # unicode category Mn (which BERT strips when do_lower_case
                # is enabled). Substituting UNK is not strictly correct, but
                # it's better than failing to return a valid parse.
                word_tokens = ["[UNK]"]
            for _ in range(len(word_tokens)):
                word_end_mask.append(0)
            word_end_mask[-1] = 1
            tokens.extend(word_tokens)
        tokens.append(u"[SEP]")
        word_end_mask.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        subword_max_len = max(subword_max_len, len(input_ids))

        all_input_ids[snum, :len(input_ids)] = input_ids
        all_word_end_mask[snum, :len(word_end_mask)] = word_end_mask

    all_input_ids = all_input_ids[:, :subword_max_len]
    all_word_end_mask = all_word_end_mask[:, :subword_max_len]
    return all_input_ids, all_word_end_mask

In [11]:
s = 'Saya sedang membaca buku tentang Perlembagaan'.split()
sentences = [s]
i, m = make_feed_dict_bert(sentences)
i, m

(array([[   3,  287,  461, 1524,  598,  454, 3809,    4]]),
 array([[1, 1, 1, 1, 1, 1, 1, 1]]))

In [12]:
charts_val, tags_val = sess.run((charts, tags), {input_ids: i, word_end_mask: m})
charts_val, tags_val

(array([[[[ 0.        , -3.4987175 , -2.3440106 , ..., -2.1679118 ,
           -2.5054266 , -2.3123686 ],
          [ 0.        , -2.3637033 , -3.160329  , ..., -1.5874335 ,
           -1.588395  , -1.3189538 ],
          [ 0.        , -1.8885597 , -2.9229689 , ..., -2.60825   ,
           -1.9523171 , -2.2531438 ],
          ...,
          [ 0.        , -2.2288191 , -4.8851027 , ..., -2.482474  ,
           -2.4275987 , -2.6169772 ],
          [ 0.        ,  0.8453851 , -2.7107687 , ..., -2.6852412 ,
           -1.9578956 , -2.507674  ],
          [ 0.        ,  0.        ,  0.        , ...,  0.        ,
            0.        ,  0.        ]],
 
         [[ 0.        , -3.1719675 , -1.8720998 , ..., -1.7905697 ,
           -1.535727  , -1.6749817 ],
          [ 0.        , -3.4987175 , -2.3440106 , ..., -2.1679118 ,
           -2.5054266 , -2.3123686 ],
          [ 0.        , -2.409241  , -2.758874  , ..., -2.593527  ,
           -2.2656188 , -2.733156  ],
          ...,
          [ 0

In [13]:
for snum, sentence in enumerate(sentences):
    chart_size = len(sentence) + 1
    chart = charts_val[snum,:chart_size,:chart_size,:]

In [14]:
# !wget https://raw.githubusercontent.com/michaeljohns2/self-attentive-parser/michaeljohns2-support-tf2-patch/benepar/chart_decoder.pyx

In [15]:
import chart_decoder_py

In [16]:
chart_decoder_py.decode(chart)

(15.76133,
 array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5]),
 array([6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6]),
 array([1, 4, 5, 0, 5, 0, 0, 3, 2, 0, 3]))

In [17]:
import nltk
from nltk import Tree

In [18]:
PTB_TOKEN_ESCAPE = {u"(": u"-LRB-",
    u")": u"-RRB-",
    u"{": u"-LCB-",
    u"}": u"-RCB-",
    u"[": u"-LSB-",
    u"]": u"-RSB-"}


def make_nltk_tree(sentence, tags, score, p_i, p_j, p_label):

    # Python 2 doesn't support "nonlocal", so wrap idx in a list
    idx_cell = [-1]
    def make_tree():
        idx_cell[0] += 1
        idx = idx_cell[0]
        i, j, label_idx = p_i[idx], p_j[idx], p_label[idx]
        label = LABEL_VOCAB[label_idx]
        if (i + 1) >= j:
            word = sentence[i]
            tag = TAG_VOCAB[tags[i]]
            tag = PTB_TOKEN_ESCAPE.get(tag, tag)
            word = PTB_TOKEN_ESCAPE.get(word, word)
            tree = Tree(tag, [word])
            for sublabel in label[::-1]:
                tree = Tree(sublabel, [tree])
            return [tree]
        else:
            left_trees = make_tree()
            right_trees = make_tree()
            children = left_trees + right_trees
            if label:
                tree = Tree(label[-1], children)
                for sublabel in reversed(label[:-1]):
                    tree = Tree(sublabel, [tree])
                return [tree]
            else:
                return children

    tree = make_tree()[0]
    tree.score = score
    return tree

In [20]:
tree = make_nltk_tree(s, tags_val[0], *chart_decoder_py.decode(chart))
print(str(tree))

(S
  (NP-SBJ (<START> Saya))
  (VP
    (PRP sedang)
    (VP
      (MD membaca)
      (NP (VB buku))
      (PP (NN tentang) (NP (IN Perlembagaan))))))


In [ ]:
def make_str_tree(sentence, tags, score, p_i, p_j, p_label):
    idx_cell = [-1]
    def make_str():
        idx_cell[0] += 1
        idx = idx_cell[0]
        i, j, label_idx = p_i[idx], p_j[idx], p_label[idx]
        label = LABEL_VOCAB[label_idx]
        if (i + 1) >= j:
            word = sentence[i]
            tag = TAG_VOCAB[tags[i]]
            tag = PTB_TOKEN_ESCAPE.get(tag, tag)
            word = PTB_TOKEN_ESCAPE.get(word, word)
            s = u"({} {})".format(tag, word)
        else:
            children = []
            while ((idx_cell[0] + 1) < len(p_i)
                and i <= p_i[idx_cell[0] + 1]
                and p_j[idx_cell[0] + 1] <= j):
                children.append(make_str())

            s = u" ".join(children)
            
        for sublabel in reversed(label):
            s = u"({} {})".format(sublabel, s)
        return s
    return make_str()

In [ ]:
make_str_tree(s, tags_val[0], *chart_decoder.decode(chart))